In [1]:
import warnings
import pandas as pd
import numpy as np

from tqdm.autonotebook import tqdm
from prophet import Prophet
from utils import create_holidays, suppress_stdout_stderr

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

DATA_PATH = 'data/prepared'
SUBMISSION_PATH = 'data/submission'

In [3]:
# Import pickled data
train_df = pd.read_pickle(f'{DATA_PATH}/train_df.pkl')
test_df = pd.read_pickle(f'{DATA_PATH}/test_df.pkl')
holidays_df = pd.read_pickle(f'{DATA_PATH}/holidays_df.pkl')

In [4]:
# Dates for prophet
train_df['Year'] = pd.DatetimeIndex(train_df.Date).year
train_df['Month'] = pd.DatetimeIndex(train_df.Date).month
train_df['Day'] = pd.DatetimeIndex(train_df.Date).day

In [5]:
def prophet_predict(store_id, train, test, 
                    holidays: pd.DataFrame = None, 
                    xreg: list = None, 
                    periods: int = 48):
    """
    Fit a prophet model and make predictions for a single store
    """
    # Select a single store per model
    train = train[train['Store'] == store_id]
    train = train.rename(columns={'Date': 'ds',
                                  'Sales': 'y'})
    train = train.sort_values('ds')
    
    model = Prophet(holidays=holidays)

    # External regressors
    if xreg is not None:
        for x in xreg:
            model.add_regressor(x)

    # Fit the model
    with suppress_stdout_stderr():
        model.fit(train)

    # Make a future dataframe with external predictions
    future = model.make_future_dataframe(periods=periods,
                                         freq='D',
                                         include_history=False)
    if xreg is not None:
        for x in xreg:
            future[x] = test[x]

    # ...and predict on it
    forecast = model.predict(future)
    forecast = forecast[['ds','yhat']].rename(columns={'ds':'Date', 
                                                       'yhat':'Sales'})
    forecast['Store'] = store_id

    return forecast

In [6]:
# Select external regressors for prophet
include_list = ['Open', 'Promo', 'CompetitionDistance', 
                'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 
                'Promo2', 'Promo2SinceWeek',
                'Promo2SinceYear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth']
xreg_list = [x for x in train_df.columns.values.tolist() if x in include_list]

In [7]:
# Train a univariate prophet model for each store and make predictions
stores_test = test_df['Store'].unique()

submission_df = []
for store in tqdm(stores_test):
    pred = prophet_predict(store_id=store, 
                           train=train_df, 
                           test=test_df, 
                           holidays=holidays_df, 
                           xreg=xreg_list)
    submission_df.append(pred)

submission_df = pd.concat(submission_df)

100%|██████████| 856/856 [28:02<00:00,  1.97s/it]


In [8]:
# Prepare for submission
submission_df = pd.merge(submission_df, test_df, on=['Store', 'Date'])
submission_df = submission_df[['Id', 'Sales']]

# Quick check
assert submission_df.shape[0] == test_df.shape[0]

submission_df.to_csv(f'{SUBMISSION_PATH}/submission_prophet.csv', index=False)